In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
#from tensorflow import keras
#from tensorflow.keras import layers
import pandas as pd

import keras
import keras.backend
from keras import layers
from keras import models
import keras.utils

print(tf.__version__)

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import math
import time
import h5py
import sklearn
from sklearn.utils import shuffle

import os

Using TensorFlow backend.


1.14.0


Import data from RNN_data.npz

In [2]:
# here we input the dummy data I haven't made yet

In [3]:
grid = 32
# training data : 10,000 signal samples and 10,000 bkg samples
traindata_sig = np.load('data/dummyvectors_signal.npz')
traindata_bkg = np.load('data/dummyvectors_bkg.npz')
trainlabels = np.concatenate((traindata_sig['labels'],traindata_bkg['labels']),axis=0)
train
# testing data : 10,000 signal samples and 10,000 bkg samples
testdata_sig = np.load('data/jetImages_signal_test.npz')
testdata_bkg = np.load('data/jetImages_bkg_test.npz')
testlabels = np.concatenate((testdata_sig['labels'], testdata_bkg['labels']),axis=0)
trainradii = np.concatenate((traindata_sig['radiilab'], traindata_bkg['radiilab']), axis = 0)
traintheta = np.concatenate((traindata_sig['thetalab'], traindata_bkg['thetalab']), axis = 0)
testradii = np.concatenate((testdata_sig['radiilab'], testdata_bkg['radiilab']), axis = 0)
test_theta = np.concatenate((testdata_sig['thetalab'], testdata_bkg['thetalab']), axis = 0)
trainz = np.concatenate((traindata_sig['zlab'], traindata_bkg['zlab']), axis = 0)
testz = np.concatenate((testdata_sig['zlab'], testdata_bkg['zlab']), axis = 0)
n_train = len(trainlabels)
n_test = len(testlabels)

trainimages = trainimages.reshape(n_train, grid, grid, 1)
testimages = testimages.reshape(n_test, grid, grid, 1)

FileNotFoundError: [Errno 2] No such file or directory: 'data/dummyvectors_signal.npz'

In [ ]:
def build_model():
    input1 = layers.Input(shape = (grid, grid, 1), name = 'image_input')
    x = layers.Conv2D(32, (11, 11), activation='relu', name = 'convd2d_1')(input1)
    x = layers.Conv2D(32, (3, 3), activation='relu', name = 'convd2d_2')(x)
    x = layers.MaxPool2D((2, 2), name = 'maxpool2d_1')(x)
    x = layers.Conv2D(32, (3, 3), activation='relu', name = 'convd2d_3')(x)
    x = layers.Conv2D(32, (3, 3), activation='relu', name = 'convd2d_4')(x)
    x = layers.MaxPool2D((2, 2), name = 'maxpool2d_2')(x)
    x1 = layers.Flatten(name = 'flatten')(x)
    input2 = layers.Input(shape=(1,), name = 'z_input')
    input3 = layers.Input(shape=(2,), name = 'radii_input')
    input4 = layers.Input(shape=(1,), name = 'theta_input')
    x = layers.concatenate(inputs = [x1, input2, input3, input4],axis=-1, name = 'concat')
    x = layers.Dense(64, activation=tf.nn.relu, name = 'relu1')(x)
    x = layers.Dense(64, activation=tf.nn.relu, name = 'relu2')(x)
    output = layers.Dense(2, activation=tf.nn.softmax, name = 'softmax')(x)
    model = models.Model(inputs=[input1, input2, input3, input4], outputs=output)
    model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['categorical_crossentropy', 'accuracy'])
    return model